# Config stuff

In [1]:
from numpy.ma.core import inner

import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

# Start local cluster

In [2]:
spark = cc.startLocalCluster("FACT_RIDES")
spark.getActiveSession()

# Make tables available

In [3]:
#EXTRACT
# Dimension date
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")

# Dimension weather
dim_weather = spark.read.format("delta").load("spark-warehouse/dimweather")

# Fact rides
fact_rides = spark.read.format("delta").load("spark-warehouse/factrides")

# Make tempviews from tables
dim_date.createOrReplaceTempView("dimDate")
dim_weather.createOrReplaceTempView("dimWeather")
fact_rides.createOrReplaceTempView("factRides")

# Analysis queries

## Student 1

### Wat zijn de drukke momenten (op dagbasis) in de week t.o.v. het weekend?

In [18]:
spark.sql("select dd.DayOfWeekStartMonday, count(fr.count_MV) as CountedRides from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by dd.DayOfWeekStartMonday \
          order by dd.DayOfWeekStartMonday asc ").show()

+--------------------+------------+
|DayOfWeekStartMonday|CountedRides|
+--------------------+------------+
|                   1|      677374|
|                   2|      717198|
|                   3|      513832|
|                   4|      403862|
|                   5|      398042|
|                   6|      511975|
|                   7|      701441|
+--------------------+------------+



In de week zijn vooral maandag en dinsdag de drukste dagen. Daarna neemt het aantal rides af elke dag.
In het weekend is het op zijn drukst op zondag.

### Hebben datumparameters invloed op de afgelegde afstand?

### Heeft weer invloed op ritten?

Op welke eigenschappen van de ritten kan het weer een invloed hebben:
1) Op aantal afgelegde ritten in een bepaald weertype
2) Op de gemiddelde duurtijd van een rit in een bepaald weertype
3) Op de gemiddelde afgelegde afstand van een rit in een bepaald weertype

In [30]:
spark.sql("select dw.weather_type, count(count_MV) as CountedRides, \
          round(sum(rideDuration_MV)/count(count_MV)) as AverageRideDuration \
          from factRides fr \
          inner join dimWeather dw on fr.weather_SK = dw.weather_SK \
          group by dw.weather_type \
          having dw.weather_type != 'weertype onbekend'").show()

+------------+------------+-------------------+
|weather_type|CountedRides|AverageRideDuration|
+------------+------------+-------------------+
|    Neutraal|         208|              677.0|
|   Aangenaam|         197|              718.0|
| Onaangenaam|          90|              635.0|
+------------+------------+-------------------+



Op basis van de analyse hierboven zouden we kunnen zeggen dat bij neutraal of aangenaam weer er het meeste fietsritten worden afgewerkt en dat ook de fietstijd langer is. Bij onaangenaam weer zijn er duidelijk minder ritten die worden afgewerkt en ook de fietstijd is het kortst van allemaal.

In praktijk weten we echter dat we random weer situaties hebben toegevoegd aan onze data. Dus de patronen die we hier zijn zijn louter toeval.

### Extra vraag 1

### Extra vraag 2

## Student 2

### Wat is de invloed van de woonplaats van de gebruikers op het gebruik van de vehicles?

### We willen voorspellen welke sloten preventief onderhoud nodig hebben. Bekijk hoe vaak slotnummers relatief gezien gebruikt worden.

### Als een klant zijn abonnement stopzet, willen we kunnen voorspellen op welke stations dit het meeste effect zal hebben.

### Extra vraag 1

### Extra vraag 2